In [9]:
import re
from tqdm import tqdm
import numpy as np
import random
from pathlib import Path
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

In [21]:
path_to_data = "/home/kirrog/projects/CheckerApp/dataset/lenta-ru-news.csv"

In [27]:
def cut_text(text: str):
    d = ""
    try:
        d = " ".join(text[max(text.index(",\"") + 2, text.index("/,") + 2):text.index("\",")].split())
    except Exception as e:
        print(e)
    return d

In [23]:
with open(path_to_data, "r") as f:
    data_l = f.read().split("https://")

In [28]:
r = []
for l in data_l:
    d = cut_text(l)
    if d != "":
        r.append(d)

substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found
substring not found


In [32]:
len(data_l)

TypeError: object of type 'NoneType' has no len()

In [31]:
data_l = None

In [30]:
len(r)

799742

In [33]:
r[0]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. «Русский инвалид», 16 сентября 1914 года.'

In [35]:
r = [re.sub('\n', f"", x) for x in r]

In [2]:
clean_text_path = "/home/kirrog/projects/CheckerApp/dataset/lenta-ru-news-only-text.txt"

In [36]:
with open(clean_text_path, "w") as f:
    for line in r:
        f.write(f"{line}\n")

In [23]:
with open(clean_text_path, "r") as f:
    r = f.read().split("\n")

In [35]:
r[0]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. «Русский инвалид», 16 сентября 1914 года.'

In [29]:
string_russian = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
augment_rate = 0.1


def augment_text(text: str):
    new_words = []
    words = text.split(" ")
    for w in words:
        if random.random() < augment_rate:
            pos = int((len(w) - 1) * random.random())
            try:
                if w[pos] in string_russian:
                    nw = w[:pos] + string_russian[int((len(string_russian) - 1) * random.random())] + w[pos + 1:]
                    w = nw
            except Exception as e:
                print(f"pos: {pos} word: {w} text: {text}")
        new_words.append(w)
    return " ".join(new_words)

In [36]:
data_augment = [augment_text(x) for x in tqdm(r)]

100%|██████████| 799743/799743 [00:24<00:00, 32275.28it/s]


In [2]:
augment_text_path = "/home/kirrog/projects/CheckerApp/dataset/lenta-ru-news-augment-text.txt"

In [39]:
with open(augment_text_path, "w") as f:
    for line in data_augment:
        f.write(f"{line}\n")

In [3]:
with open(augment_text_path, "r") as f:
    data_augment = f.read().split("\n")

In [11]:
def slit_to_train_test_valid(data):
    tr, te = train_test_split(data, test_size=0.30, shuffle=False)
    test, valid = train_test_split(te, test_size=0.33, shuffle=False)
    return tr, test, valid

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [13]:
bert_tokens = {
    'START_SEQ': 101,
    'PAD': 0,
    'END_SEQ': 102,
    'UNK': 100
}

In [19]:
from typing import List


def fill_matrix_with_tokens(texts: List[str]) -> np.array:
    matrix = np.zeros((len(texts), 768), dtype=np.int32)
    counter = 0
    inp_size = 768
    for i, text_ in tqdm(enumerate(texts), desc="tokenizing", total=len(texts)):
        tokens = [bert_tokens["START_SEQ"]] + tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_)) + [
            bert_tokens["END_SEQ"]]
        threshold = min(inp_size, len(tokens))
        if threshold > inp_size:
            counter+=1
        matrix[i, 0: threshold] = tokens[:threshold]
    print(f"Cutted: {counter}")
    return matrix


In [15]:
def save_as_train_test_valid_data(dir, prefix, data):
    train, test, valid = slit_to_train_test_valid(data)
    train_path = Path(dir) / (prefix + "_train.npy")
    test_path = Path(dir) / (prefix + "_test.npy")
    valid_path = Path(dir) / (prefix + "_valid.npy")
    np.save(str(train_path), fill_matrix_with_tokens(train))
    np.save(str(test_path), fill_matrix_with_tokens(test))
    np.save(str(valid_path), fill_matrix_with_tokens(valid))

In [ ]:
save_as_train_test_valid_data("/home/kirrog/projects/CheckerApp/dataset", "clean", r)

In [10]:
len(data_augment)

799744

In [17]:
data_augment[0]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийхком бою принимают участие тяжелые калигры. С раннего утра 14 сентябня огонь достиг значительного нхпряжения. Попытка германской пехоты пюобиться блаже к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая жо шоссе цт Перемышля к Саноку, обстреливалась с вйсот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новйе партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артчллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. «Русский инвалид», 16 сентября 1914 года.'

In [20]:
save_as_train_test_valid_data("/home/kirrog/projects/CheckerApp/dataset", "aug", data_augment)

tokenizing: 100%|██████████| 559820/559820 [05:39<00:00, 1648.81it/s]


Cutted: 0


tokenizing: 100%|██████████| 160749/160749 [02:38<00:00, 1011.41it/s]


Cutted: 0


tokenizing: 100%|██████████| 79175/79175 [01:24<00:00, 934.91it/s] 


Cutted: 0


In [ ]:
d = np.zeros((10, 10), dtype=np.uint16)

In [ ]:
!pip install labml == 0.4.125

In [ ]:
!pip install transformers == 4.7.0

In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
!cd / content / drive / MyDrive / punct & python / content / drive / MyDrive / punct / train.py \
    --model-name repunct-model \
    --pretrained-model DeepPavlov / rubert-base-cased-sentence \
    --targets S CM D EL EX Q CL \
    --train-data / content / drive / MyDrive / punct / lenta_res_train.tsv \
    --val-data / content / drive / MyDrive / punct / lenta_res_valid.tsv \
    --test-data / content / drive / MyDrive / punct / lenta_res_test.tsv \
    --store-best-weights \
    --epoch 7 \
    --batch-size 4 \
    --augment-rate 0.15 \
    --labml \
    --seed 1 \
    --cuda \
    --save-dir / content / drive / MyDrive / punct / models